In [1]:
import pandas as pd
import  tca_cx as tca
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import *
import numpy
from kcca import *
from numpy import dot, eye, ones, zeros
import scipy.linalg
from kernel_icd import kernel_icd
from kernels import *
import pandas as pd
from plot_confusion_matrix import *
from sklearn.cluster import KMeans as KM

In [2]:
import tensorflow as tf

c:\users\10151\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\10151\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\10151\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\10151\appd

In [3]:
tf.__version__

'1.7.0'

In [2]:
train17=pd.read_csv('20170529_standard.csv')
train17_sample=train17.sample(frac=1,replace=True, random_state=3, axis=0)
train18=pd.read_csv('20180509_standard.csv')
train18_sample=train18.sample(frac=1,replace=True, random_state=3, axis=0)
#train18_sample

In [3]:
train_sample=pd.concat([train17_sample,train18_sample], axis=1,ignore_index=False)
#train_sample

In [4]:
#分割原域和目标域的训练数据和标签数据
S=train17_sample.loc[:,'S1':'S4'].values
#S.info()
nml=Normalizer().fit(S) 
S=nml.transform(S).astype("float32")
S_Y=train17_sample.loc[:,'S_Y'].values
T=train18_sample.loc[:,'T1':'T4'].values
T=nml.transform(T).astype("float32")
T_Y=train18_sample.loc[:,'T_Y'].values

In [5]:
kernel =GaussianKernel()
cca = KCCA(kernel, kernel,
                regularization=1e-5,
                decomp='full',
                method='kettering_method',
                scaler1=None,
                scaler2=None).fit(S,T)     
X1, X2 = cca.transform(S, T)
dX=X1-X2 #kcca 的MAD变量
km=KM(n_clusters=2)#无监督二分类 得出0或1是否改变
pre=km.fit_predict(X=dX)
pre=pd.DataFrame({"changed":pre})

c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [6]:
train_sample=train_sample.reset_index().drop(['index'],axis=1)#删除索引
tag_train_sample=pd.concat([train_sample,pre], axis=1,ignore_index=True)
tag0_train_sample=tag_train_sample[tag_train_sample[10] == 0]
tag1_train_sample=tag_train_sample[tag_train_sample[10] == 1]
tag_train_sample=tag0_train_sample

In [7]:
Xs = tag_train_sample.iloc[:,:4]
Xs = Xs.values
Ys = tag_train_sample.iloc[:,4:5]
Ys = Ys.values
Xt = tag_train_sample.iloc[:,5:9]
Xt = Xt.values
Yt = tag_train_sample.iloc[:,9:10]
Yt = Yt.values
X = np.hstack((Xs,Xt))
Y = np.hstack((Ys,Yt))
x_train_st, x_test_st, y_train_st, y_test_st = train_test_split(
    X,
    Y,
    test_size=0.5, 
    random_state=9)
x_train_s=x_train_st[:,0:4]
y_train_s=y_train_st[:,0]
x_train_t=x_train_st[:,4:8]
y_train_t=y_train_st[:,1]
x_test_t=x_test_st[:,4:8]
y_test_t=y_test_st[:,1]

In [8]:
my_tca = tca.TCA(dim=6)
predict,acc_tar_o,y_tar_o_pre,kappa_tar = my_tca.classify_svm(Xs,Ys,Xt,Yt)
print('SVM',acc_tar_o,kappa_tar)
#plot_confusion_matrix(y_train_t, predict,("0",'1','2',"3",'4','5'))
#plot_confusion_matrix(y_train_t, predict,("0",'1','2',"3",'4','5','6'))

SVM 0.692200557103064 0.568478226137066


c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
#plot_confusion_matrix(y_train_t, predict,("0",'1','2',"3",'4'))

In [11]:
x_src_tca, x_tar_tca, x_tar_o_tca = my_tca.fit_transform(Xs, Xt,Xt) 
predict,acc_tar_o,y_tar_o_pre,kappa_tar = my_tca.classify_svm(x_src_tca, Ys, x_tar_o_tca, Yt)
print('SVM',acc_tar_o,kappa_tar)
#plot_confusion_matrix(Yt, predict,("0",'1','2',"3",'4','5','6'))

c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


SVM 0.24651810584958217 0.09628644730956903


 👆👆👆👆👆👆👆